## Exercice I. Rejet Optimisé

Le but de cet exercice est de sumuler une loi normale $\mathcal{N(0,1)}$ en  utilisant comme proposition une loi de Laplace de paramètre $\lambda > 0$, c'est-à-dire de densité
$$g(x) = \frac{\lambda}{2} e^{-\lambda |x|}$$

### &#8903; I.1. Simulation d'une loi de Laplace
Pour appliquer cet algorithm de rejet, il faut donc commencer par simuler $Y$ suivant une loi de Laplace. Ecrivez une fonction __rlaplace(n,lambda)__ qui donne un échantillons i.i.d. $Y$ (taille = $n$).

On a deux méthodes classiques pour simuler la loi de Laplace.

#### I.1.1 Méthode d'inversion

Partant de la fonction de répartition

$$
F(y) = \mathbf{1}_{\{y < 0 \}}\cdot e^{\lambda y} 
+ \mathbf{1}_{\{ y \geq 0\}} \cdot(1 - \frac12 e^{-\lambda y})
$$

on en déduit son inverse généralisée

$$
F^{-1} (u) = \mathbf{1}_{\{ 0 < u <\frac12  \}}\cdot\frac1{\lambda}\log (2u) 
- \mathbf{1}_{\{\frac12 \leq u < 1\}} \cdot \frac1{\lambda}\log\big(2(1-u)\big) 
$$

Il suffit de tirer $U \sim \mathcal{U}_{[0,1]}$ puis de poser $Y = F^{-1} (U)$

In [ ]:
# Solution:
rlaplace_inv <- function (n, lambda){
    
    rY <- function(){
        U <- runif(1)
        if(U < .5){
            return(1./lambda*log(2*U))
        }
        else{
            return( - 1./lambda*log(2*(1-U)))
        }
    }
    return (replicate(n, rY()))
}

#### I.1.2  Distribution de mélange

On peut aussi voir la loi de Laplace comme un mélange équiprobable d'une exponentielle positive d'une exponentielle négative. 
Ainsi, une méthode de simulation est de commencer par tirer une Rademacher $R$, c'est-à-dire, $R = \pm 1$ de façon équiprobable.

$$
R = -\mathbf{1}_{\{ U \leq \frac12 \}} + \mathbf{1}_{\{ U > \frac12 \}}
\qquad avec\;U\sim \mathcal{U}_{[0,1]}
$$

On tire ensuit une loi $E \sim \mathcal{E}(\lambda)$ exponentielle de paramètre $\lambda$ pour obtenir $Y = R \times E$. 

In [ ]:
# Solution:
rlaplace_mel <- function(n, lambda){
    
    rY <- function(){
        U = runif(1)
        R <- -(U < .5) + (U >= .5)
        return(rexp(1,lambda) * R)
    }
    return (replicate(n, rY()))
    
}

Comparez les histogrammes avec la vraie densité pour $\lambda = 2$ et reproduire le graphe suivant
![image](exo_1_1.png)

In [ ]:
# Solution:
# png('exo_1.png',width = 750)
# inv = rlaplace_inv(10000,2)
# mel = rlaplace_mel(10000,2) 
# dlaplace <- function(x) return(exp(- 2*abs(x)))
# par(mfrow = c(1,2))
# hist(inv,probability = T, main = "Méthode d'inverse", xlab = 'Y', ylim = c(0, 1), breaks = 50)
# curve(dlaplace(x), add = T)
# legend('topright',c('density'), lty = c(1))
# hist(mel,probability = T, main = "Distribution de mélange", xlab = 'Y', ylim = c(0, 1), breaks = 50)
# curve(dlaplace(x), add = T)
# legend('topright',c('density'), lty = c(1))
# dev.off()

Vous pouvez choisir __`rlaplace_inv`__ ou __`rlaplace_mel`__ pour définir la fonction __rlaplace__ comme vous voulez pour la suite.

In [ ]:
rlaplace <- function(n, lambda){
    return(rlaplace_mel(n,lambda))
}

### &#8903; I.2 Paremètre optimisé 
Déterminer la valeur de $\lambda^*$ qui permet de minimiser la probabilité de rejet. Vérifier la probabilité de rejet minimisée vaut environ 0,24. 

(la preuve n'est pas demandée. Par contre, la formule pour calculer cette probabilité est nécéssaire.)

Avec $\lambda^*$, évcrivez une fonction __rnorm_ro(n)__ qui donne un échantillon i.i.d. de $\mathcal{N}(0,1)$ (taille = $n$).

In [ ]:
# Solution:
m <-  1 - sqrt(pi / (2*exp(1)))
print(paste('probabilité de rejet minimisée: ', m))

rnorm_ro <- function(n){
    # initiation
    ratio <- function(x){
        return (sqrt(2/pi) * exp(abs(x) - x^2*.5))
    }
    k <- 0
    ech = c()
    # On utilise while pour contrôler la taille d'échantillon.
    while(k < n){
     
        Y <- rlaplace(1,1)
        U <- runif(1)
        if(U <= ratio(Y)){
            ech <- c(ech,Y)
            k <- k+1
        }
    }
    return(ech)
}

Comparez avec la vraie densité d'une loi normale centrée réduite et reproduire le graphe suivant.
![image](exo_1_2.png)

In [ ]:
# Solution:
# png('exo_1_2.png',width = 750)
# hist(rnorm_ro(1000),probability = T, breaks = 30, 
#      main = "Simulation par la méthode rejet", xlab = 'X')
# curve(dnorm(x),add=T)
# legend('topright', c('density'), lty = c(1))
# dev.off()

### &#8903; I.3 Application: Naive Monte Carlo 

Estimer la valeur numérique de l'intégrale
$$
I  = \int_{\mathbb{R}} \frac{(x^2 + y^2)}{\sqrt{2\pi} } \cdot \exp \big\{- \big(x^2 + 2\vert y \vert \;\big)\big\}
dxdy
$$
en utilisant les fonctions __rlaplace__ et __rnorm_ro__ avec intervalle de confiance de nivea 95% (asymptotic) donnée par TCL.

Reproduire le graphe suivant.
![image](exo_1_3.png)

In [ ]:
# Solution:

# png('exo_1_3.png',width = 750)
# numsim <- 5000
# x <- rnorm_ro(numsim)
# y <- rlaplace(numsim,lambda = 2)
# f <- function (x,y) return((x^2+y^2))
# m <- cumsum(f(x,y)) / 1:numsim                   
#                                               
# plot(m, col = "red", type = 'l',main = 'Estimation de I_n par M.C.', xlab = 'n', ylab = 'I_n')                           
# 
# v <- sapply(2:numsim, function(k) var(x[1:k]))  
# v <- c(NA, v)                                   
# e <- 1.96 * sqrt(v) / sqrt(1:numsim)            # taille de l'IC à 95%
# lines(m+e, lty = 2)
# lines(m-e, lty = 2)
# legend('topright', c('estimation','IC 95%'), lty = c(1,2), col = c('red','black'))
# grid()
# dev.off() 

## Exercice II. Inférence Bayésienne par MCMC 

Soit $(X,Y)^T$ un vecteur gaussien 
$$
(X,Y)^T \sim \mathcal{N}_2 \big( 0 , \Sigma_{2\times 2} \big) 
$$
avec 
$$
\Sigma_{2\times 2} = \begin{pmatrix} 1 & \rho \\ \rho & 1\end{pmatrix}
$$


### &#8903; II.1. Simulation des données 

Simuler __ech_test__ un échantillon de $(X,Y)^T$ de taille __N__ = 500  avec $\rho = 0.6$ (cf. TP2)

In [ ]:
# Soluition:
N_test <- 100
rho_test <- 0.6

rG <- function(n = 2, rho = rho_test){
    G_0 <- rnorm(1)
    ech <- sqrt(rho) * G_0 + sqrt(1 - rho) * rnorm(n)
    return(ech) 
}  
ech_test = replicate(N_test, rG())


### &#8903; II.2. Modèle Bayésien 

Comme la taill d'échantillon __N_test__ est relativement petite, on considère alors une inférence bayésienne.
Maintenant, on suppose que $\rho$ est aussi une variable aléatoir, qui suit une loi a priori de Jeffreys. C'est-à-dire, de densité 
$$
\pi(\rho) = \frac1{(1-\rho^2)^{3/2}}\cdot \mathbf{1}_{[0,1]}(\rho)
$$

Alors, la fonction de vraisemblance $\mathcal{L}(X_{1:N},Y_{1:N}\mid \rho): \rho \rightarrow \mathbb{R}_{+}$ est de forme

$$
\mathcal{L}(X_{1:N},Y_{1:N}\mid \rho) = \prod_{i=1}^N 
\frac1{2\pi\sqrt{1-\rho^2}} 
\exp \big\{ - \frac1{2(1-\rho^2)} \big(X_i^2 - 2\rho X_i Y_i + Y_i^2 \big)\big\}
$$
où $(X_{1:N},Y_{1:N})^T =$ __ech_test__ les données qu'on a simulées.

Le but de cet exercice est de conduire une inférence bayésienne sur la loi a postériori $\pi(\rho' \mid X_{1:N},Y_{1:N})$. 

D'après la formule de Bayes, on a 
$$
\pi(\rho' \mid X_{1:N},Y_{1:N})\propto 
f(\rho' \mid X_{1:N},Y_{1:N}) 
$$
avec
$$
f(\rho' \mid X_{1:N},Y_{1:N}) =
\frac1{(1-{\rho}'^2)^{3/2}}\cdot \mathbf{1}_{[0,1]}(\rho')
\prod_{i=1}^N 
\frac1{\sqrt{1-{\rho'}^2}} 
\exp \big\{ - \frac1{2(1-{\rho'}^2 )} \big(X_i^2 - 2\rho' X_i Y_i + Y_i^2 \big)\big\}
$$

Vu que la forme est assez compliquée, on considère donc la méthode MCMC pour simuler $\rho'$ suivant la loi a postériori $\pi(\rho' \mid X_{1:N},Y_{1:N})$.



### II.3 Algorithme de Métropolis en $\mathbb{R}$

On considère une transition uniform $P_{\lambda}(x,dy)$ sur l'intervalle $[x-\lambda,x+\lambda]$.  
Plus précisement,
$$P_{\lambda}(x,dy) = P_{\lambda} (x,y) dy = \frac1{2\lambda}\mathbf{1}_{\{ y\in [x-\lambda,x+\lambda]\}}dy$$
où $\lambda \in \mathbb{R}_+$. 
Pour simplifier le problème, on commence par $\lambda = 0.1$.

Le ratio de Métropolis $r(x,y)$ est de la forme 
$$
r(x,y) = \frac{P_{\lambda} (y,x)f(y\mid X_{1:N},Y_{1:N})}{P_{\lambda} (x,y)f(x\mid X_{1:N},Y_{1:N})} \wedge 1
$$
Ici, comme la loi $P_{\lambda}$ est symétrique, on a
$$
r(x,y) = \frac{f(y\mid X_{1:N},Y_{1:N})}{f(x\mid X_{1:N},Y_{1:N})} \wedge 1
$$


##### Simulation d'une chaine de markov $(\xi_p)_{p\geq 0}$ qui admet $\pi(\cdot\mid X_{1:N},Y_{1:N})$ comme une loi invariante

* Initialisation ($p=0$)

    Simuler $\xi_0\sim \mathcal{U}_{[0,1]}$  
    
* Dynamique de Métropolis ($p = 1,2,\dots$)
    1. En sachant $\xi_{p-1}$, on propose un candidat $\hat{\xi}_p$, tel que
    $$\hat{\xi}_p \sim P_{\lambda}(\xi_{p-1},dy)$$
    
    2. Calculer $r(\xi_{p-1},\hat{\xi}_p)$ 
    
    3. On simule $U \sim \mathcal{U}_{[0,1]}$. 
    
        - Si $U \leq r(\xi_{p-1},\hat{\xi}_p)$, on prend $\xi_p = \hat{\xi}_p$ 
        - Si $U > r(\xi_{p-1},\hat{\xi}_p)$, on prend $\xi_p = \xi_{p-1}$ 
        
    


#### II.3.1 Ratio de Métropolis 

Ecrivez une fonction __ratio_MP(x, y, ech)__ qui donne la valeur 
$$\frac{f(y\mid X_{1:N},Y_{1:N})}{f(x\mid X_{1:N},Y_{1:N})}$$ 
sachant __ech__$=(X_{1:N},Y_{1:N})$ avec convention $\frac{0}{0} = 0$.

In [ ]:
# Solution:
ratio_MP <- function(x,y,ech = ech_test){
        N <- length(ech[1,])
        result <- sqrt((1-x^2)^3/(1-y^2)^3)
    
        for (i in 1:N) {
            result <- result * (sqrt(1-x^2))/(sqrt(1-y^2)) 
            result <- result * exp(1./(2*(1-x^2)) * ( sum(ech[,i]^2) - 2 * x * ech[1,i]*ech[2,i]) - 1./(2*(1-y^2)) * ( sum(ech[,i]^2) - 2 * y * ech[1,i]*ech[2,i]))
        } 
        if((x<0)*(x>1)*(y<0)*(y>1) == 1) return (0)
                                                                     
        return(result)    
    
}

#### II.3.2 Simulation de la loi a postériori 
Écrivez une fonction __post_mcmc(n, ech)__ qui donne un échantillons d'une chaine $(\xi_p)_{0\leq p \leq n}$ qui admet la loi a postériori $\pi(\cdot \mid X_{1:N},Y_{1:N})$ comme la loi invariante. 

Puis, simuler un échantillons __ech_post__ de taille __N_sim = 5000__. 

In [ ]:
# Solution:
post_mcmc <- function(n, ech = ech_test, lambda = 0.1){
    # initiation
    xi <- c(runif(1))
    for (i in 1:n){
        xi_hat <- runif(1, min = xi[i] - lambda, max = xi[i] + lambda)
        r <- ratio_MP(xi[i], xi_hat, ech = ech)
        U <- runif(1)
        if (U <= r){
            xi <- c(xi, xi_hat)
        }else{
            xi <- c(xi, xi[i])
        }
    }
    return(xi)
}

N_sim <- 5000
ech_post <- post_mcmc(N_sim,ech = ech_test)

### II.4 Illustrations et Estimations 

![image](exo_2_1.png)

Estimer la probabilité 
$$\mathbf{P}\big(\rho' > 0.55 \mid X_{1:N},Y_{1:N}\big)$$
en utilisant __ech_post__ par la méthode Monte Carlo avec un Intervalle de confiance de niveau 95%.

In [ ]:
# Solution:
m_rho <- mean(ech_post>0.55)
e_rho <- 1.96 * sd(ech_post>0.55) / sqrt(N_sim)
print(m_rho)
print(paste('IC (95%): '))
print(c(m_rho-e_rho,m_rho+e_rho))
#png('exo_2_1.png',width = 750)
hist(ech_post, breaks =100,freq  = FALSE, main = 'Histogramme de la loi a postériori')
abline(v = 0.55,col = 'darkred', lwd = 2)
#dev.off()